In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import os
import multiprocessing
from multiprocessing import Pool
import struct
import random
import math
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy import optimize as opt
import scipy
from scipy.io import loadmat  
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
# Forked from 'https://gist.github.com/akesling/5358964' -> GPL2
def read(path):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    fname_img = os.path.join(path, 'training_set_images')
    fname_lbl = os.path.join(path, 'training_set_labels')

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)

allData = []
for i in read('data/'):
    allData +=  [ (i[1], i[0])]

In [ ]:
sampleCount = 100
data = pd.DataFrame( allData[0:sampleCount], columns=['Image','Label'] )
data.head()

In [ ]:
(width, height) = data[0:1]['Image'][0].shape

In [ ]:
temp = pd.DataFrame()
for i in range(sampleCount): 
    curr = data['Image'][i] / 255.
    temp = temp.append( pd.DataFrame(curr.reshape((1, width*height))) )

temp.insert(0, 'Ones', 1)
temp.head()

In [ ]:
x = temp
y = data['Label']

# Visualizing the Data

In [ ]:
(row, col) = (2, 5)

fig, axes = plt.subplots(nrows=row, ncols=col)
plt.figure(figsize=(16,9))

for i in range(row):
    for j in range(col):
        currGen = allData[ random.randint(0, len(data)-1) ]
        axes[i][j].imshow(currGen[0], cmap=mpl.cm.GnBu, interpolation="bicubic")
        axes[i][j].set_title( str(currGen[1]) )
        
        axes[i][j].xaxis.label.set_visible(False)
        axes[i][j].axes.get_xaxis().set_visible(False)
        axes[i][j].axes.get_yaxis().set_visible(False)

In [ ]:
# # pd.DataFrame(data[0:6000])
# data = np.array(data)
# data

# Logistic Regression
### sigmoid function
$f(x)=\frac{1}{1+e^{-x}}$

In [ ]:
def sigmoid(z):  
    return 1 / (1 + np.exp(-z))

tempX = np.array(range(-10,11))
tempY = list( map(sigmoid, tempX) )
plt.plot(tempX, tempY, 'b')

In [ ]:
def cost(theta, x, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(x)
    y = np.matrix(y)
    left = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    right = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = ( learningRate / (2 * len(X)) ) * np.sum( np.power(theta[:,1:theta.shape[1]], 2) )
    return np.sum(left - right) / (len(X)) + reg

In [ ]:
def gradient(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)

    error = sigmoid(X * theta.T) - y

    for i in range(parameters):
        term = np.multiply(error, X[:,i])

        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learningRate / len(X)) * theta[:,i])

    return grad

In [ ]:
def workerFunc( args ):
    x, y, landa, label = args
    x0 = np.zeros( (x.shape[1],1) )
#     print( [1 if yy == label else 0 for yy in y] )
    print(label, "Starting!")
    res = opt.minimize(cost, x0,jac=gradient, method='BFGS', args=(x, [ 1 if yy == label else 0 for yy in y], landa), options={'disp': True})
    print(label, "Compelete!")
    return res.x
    return num
    
def findAllThatas(x, y, labels, landa=0.0001):
    p = Pool( len(labels) )
    return p.map(workerFunc, [ (x, y, landa, label) for label in labels ])

thetas = findAllThatas(x, y, [0,1,2,3,4,5,6,7,8,9])

In [ ]:
def passProbability(theta, image):
    tmp = np.array(image) / 255.
    return sigmoid( np.dot(tmp, theta) )

In [ ]:
PP = 0
for j in range(100):

    resP = 0
    resX = None
    for i in [(zzz, passProbability( thetas[zzz], x.iloc[j])) for zzz in range(10)]:
        if i[1] > resP:
            resP = i[1]
            resX = i[0]
    
    if y[j] == resX:
        PP += 1
        
print( PP )

In [ ]:
with open("data/training_set_labels", 'rb') as flbl:
    magic, num = struct.unpack(">II", flbl.read(8))
    lbl_test = np.fromfile(flbl, dtype=np.int8)

with open("data/training_set_images", 'rb') as fimg:
    magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img_test_view = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl_test),rows,cols)
    img_test = img_test_view.reshape(len(lbl_test),rows*cols)
    
thetas = np.load(".thetas")
img_test = img_test/255.
img_test = np.concatenate((np.ones((1,len(lbl_test))), img_test.T)).T

idx_test = random.randint(0,len(lbl_test))
plt.imshow(img_test_view[idx_test], cmap='gray_r')
plt.show()
          
ans = sigmoid(np.dot(thetas,img_test[idx_test].T))
for i in range(0,10):
    print( "predicted probability for value ",i," is: ",ans[i] )
print("---")
print("predicted value is: " , ans.argmax())
print("real value is: ",lbl_test[idx_test])